In [1]:
import csv
import json
import re

In [2]:
csvFile = r'customers.csv'
newCsvFile= r'customers_edited.csv'
jsonFile = r'customerMakeCustomer.json'
dataDict = []

readFieldnames = [
  'ID',
  'First name',
  'Last name',
  'Company name',
  'Nickname',
  'Primary?',
  'Parent ID',
  'Mobile phone',
  'Home phone',
  'Office phone',
  'Email',
  'Preferred notification method',
  'Street one',
  'Street two',
  'City',
  'State',
  'Postal code',
  'Tags',
]
writeFieldnames = [
  'First name',
  'Last name',
  'Company name',
  'Nickname',
  'Primary?',
  'Mobile phone',
  'Home phone',
  'Office phone',
  'Email',
  'Street one',
  'Street two',
  'City',
  'State',
  'Postal code',
  'Tags',
]

In [ ]:
def locationExists(row: dict, locations: list):
  l = iter(l for l in locations if l['zipcode'] == row['Postal code'])
  while next(l, None):
    if l['city'] == row['City']:
      if (l['address1'] == row['Street one'] or
          l['address1'] == row['Street two'] or
          l['address2'] == row['Street one'] or
          l['address2'] == row['Street two']):
        return True

  return False
#end locationExists

def makeLocation(row: dict):
  return {
    'address1': row['Street one'],
    'address2': row['Street two'],
    'city': row['City'],
    'state': 'FL',
    'zipcode': row['Postal Code'],
    'primary': True if row['Primary?'] == 'yes' else False
  }
#end makeLocation

def makeLocations(row: dict, locations: list):
  if locationExists(row, locations):
    return locations # Need to separate empty listings into separate csv/json

  newLocation = makeLocation(row)
  locations.append(newLocation)
  return locations
#end makeLocations


In [ ]:
def phoneExists(phoneNumber: str, phones: list):
  if phoneNumber in phones:
    return True
  
  return False
#end checkPhones


def makePhones(row: dict, phones: list):
  # Make an array of non-repeating phone numbers from _row_.
  mp = row['Mobile phone']
  hp = row['Home phone']
  op = row['Office phone']

  if not phoneExists(mp, phones):
    phones.append(mp)
  if not phoneExists(hp, phones):
    phones.append(hp)
  if not phoneExists(op, phones):
    phones.append(op)

  #   if mp:
  #     if not ...:

  #   if hp:
  #     if not mp == hp:
  #       phones.append(hp)
  #       if not mp == op:
  #         phones.append(op)
  #     elif not mp == op:
  #       phones.append(op)
  #     return phones
    
  #   if op:
  #     if not mp == op:
  #       phones.append(op)
    
  #   return phones
  
  # if hp:
  #   phones.append(hp)

  #   if op:
  #     if not hp == op:
  #       phones.append(op)

  #   return phones

  # if op:
  #   phones.append(hp)
  
  return phones
#end makePhones


In [ ]:
def createCustomerList():
  keys = ['firstname', 'lastname', 'email', 'phones', 'locations']
  customers = [dict.fromkeys(keys)]
  return customers
#end createCustomerList


def customerExists(email: str, customers: list):
  index = None
  it = iter((index, c) for index, c in enumerate(customers) if c['Email'] == email)
  return next(it, (None, None))
#end customerExists


def makeCustomer(row: dict, phones: list, locations: list):
  return {
    'firstname': row['Firstname'],
    'lastname': row['Last name'],
    'companyname': row['Company name'],
    'email': row['Email'],
    'phones': locations,
    'locations': locations,
  }
#end makeCustomer


def fillCustomerBlanks(row: dict, customer: dict):
  if not customer['firstname']:
    customer['firstname'] = row['First name']
  if not customer['lastname']:
    customer['lastname'] = row['Last name']
  if not customer['companyname']:
    customer['companyname'] = row['Company name']
  
  return customer
#end fillCustomerBlanks


def makeCustomers(row: dict, phones: list, newLocations: list, customers: list):
  email = row['Email'] if row['Email'] else None
  if not email: # Without an email, cannot identify unique customer:
    return customers

  index, c = customerExists(email, customers)
  if not index: # No customer match:
    newCustomer = makeCustomer(row, phones, locations)
    customers.append(newCustomer)
    return customers
  else: # A customer already exists:
    fn = row['Firstname']
    ln = row['Last name']
    cn = row['Company name']

    c['firstname'] = fn if not c['firstname'] else c['firstname']
    c['lastname'] = fn if not c['lastname'] else c['lastname']
    c['companyname'] = fn if not c['companyname'] else c['companyname']
    c['email'] = fn if not c['email'] else c['email']

    locations = c['locations']
    for l in locations:
      customers[index] = c

  return customers
#end makeCustomer


In [ ]:
# Test data:

rows = [
  {
    'ID': 124,
    'First name' : 'Bob',
    'Last name' : 'Han',
    'Company name': '',
    'Nickname': 'Bobby',
    'Primary?': 'no',
    'Parent ID': 235234,
    'Mobile phone': 'asdfa802354',
    'Home phone': '02390f0a9f',
    'Office phone': '',
    'Email': 'lzhang22@gmail.com',
    'Preferred notification method': 'do not notify',
    'Street one': '99 Frances Harrell Way',
    'Street two': '',
    'City': 'Tallahassee',
    'State': 'FL',
    'Postal code': '32343',
    'Tags': '',
  },
  {
    'ID': 125,
    'First name' : '',
    'Last name' : '',
    'Company name': '',
    'Nickname': 'Bobby',
    'Primary?': 'no',
    'Parent ID': 23524,
    'Mobile phone': '',
    'Home phone': '02390f0a9f',
    'Office phone': '',
    'Email': 'lzhang22@gmail.com',
    'Preferred notification method': 'do not notify',
    'Street one': '315 Mae Cato Rd',
    'Street two': '',
    'City': 'Midway',
    'State': 'FL',
    'Postal code': '32343',
    'Tags': '',
  },
  {
    'ID': 122356,
    'First name' : 'Cheryl',
    'Last name' : 'Oldani',
    'Company name': '',
    'Nickname': '',
    'Primary?': 'no',
    'Parent ID': 23529,
    'Mobile phone': '',
    'Home phone': '902309f2f',
    'Office phone': '',
    'Email': 'cheryl.oldani@gmail.com',
    'Preferred notification method': 'do not notify',
    'Street one': '75 Old Lloyd Rd',
    'Street two': '',
    'City': 'Monticello',
    'State': 'FL',
    'Postal code': '32342',
    'Tags': '',
  },
  {
    'firstname': 'Joe',
    'lastname': 'Adams',
    'companyname': '',
    'email': 'joeadamscoast@comcast.net',
    'phones': [
      '3217398',
    ],
    'locations': [
      {
        'address1': '3544 Oakhill Trail',
        'address2': '',
        'city': 'Tallahassee',
        'state': 'FL',
        'zipcode': '32312',
        'primary': True,
      },
    ],
    'locations': [
      {
        'address1': 'Old Dirt Rd',
        'address2': '',
        'city': 'Tallahassee',
        'state': 'FL',
        'zipcode': '32317',
        'primary': False,
      },
    ],
  },
  {
    'ID': 100,
    'First name': 'Bob',
    'Last name': 'Han',
    'Company name': '',
    'Nickname': '',
    'Primary?': 'no',
    'Parent ID': 101,
    'Mobile phone': '',
    'Home phone': '',
    'Office phone': '',
    'Email': 'lzhang22@gmail.com',
    'Preferred notification method': 'do not notify',
    'Street one': '415 Mae Cato Rd',
    'Street two': '',
    'City': 'Midway',
    'State': 'FL',
    'Postal code': '32343',
    'Tags': '',
  },
  {
    'ID': 102,
    'First name': 'Joe',
    'Last name': 'Adams',
    'Company name': 'Adams Quality Homes',
    'Nickname': '',
    'Primary?': 'no',
    'Parent ID': 101,
    'Mobile phone': '3197878',
    'Home phone': '3217398',
    'Office phone': '5709242',
    'Email': 'joeadamscoast@comcast.net',
    'Preferred notification method': 'do not notify',
    'Street one': '3733 Tom John Ln',
    'Street two': '',
    'City': 'Tallahassee',
    'State': 'FL',
    'Postal code': '32309',
    'Tags': '',
  },
  {
    'ID': 101,
    'First name': 'Joe',
    'Last name': 'Adams',
    'Company name': 'Adams Quality Homes',
    'Nickname': '',
    'Primary?': 'yes',
    'Parent ID': 101,
    'Mobile phone': '3197878',
    'Home phone': '3217398',
    'Office phone': '5709242',
    'Email': 'joeadamscoast@comcast.net',
    'Preferred notification method': 'do not notify',
    'Street one': '3544 Oakhill Trail',
    'Street two': '',
    'City': 'Tallahassee',
    'State': 'FL',
    'Postal code': '32312',
    'Tags': '',
  },
  {
    'ID': 101,
    'First name': 'Joe',
    'Last name': 'Adams',
    'Company name': 'Adams Quality Homes',
    'Nickname': '',
    'Primary?': 'yes',
    'Parent ID': 101,
    'Mobile phone': '3197878',
    'Home phone': '',
    'Office phone': '5709242',
    'Email': 'joeadamscoast@comcast.net',
    'Preferred notification method': 'do not notify',
    'Street one': '3544 Oakhill Trail',
    'Street two': '',
    'City': 'Tallahassee',
    'State': 'FL',
    'Postal code': '32312',
    'Tags': '',
  },
  {
    'ID': 103,
    'First name': 'Joe',
    'Last name': 'Adams',
    'Company name': 'Adams Quality Homes',
    'Nickname': '',
    'Primary?': 'no',
    'Parent ID': 101,
    'Mobile phone': '3197878',
    'Home phone': '3217398',
    'Office phone': '5709242',
    'Email': 'joeadamscoast@comcast.net',
    'Preferred notification method': 'do not notify',
    'Street one': '2070 Solomon Ct',
    'Street two': '',
    'City': 'Tallahassee',
    'State': 'FL',
    'Postal code': '32312',
    'Tags': '',
  },
]


In [ ]:
# with open(csvFile, newline='') as input, open(newCsvFile, 'w', newline='') as output:
#   csvReader = csv.DictReader(input)
#   csvWriter = csv.DictWriter(output)
#   csvWriter.writeheader()

#   for row in csvReader:
#     fn = row['First name']
#     ln = row['Last name']
#     cn = row['Company name']
#     nn = row['Nickname']

#     mp = row['Mobile phone']
#     hp = row['Home phone']
#     op = row['Office phone']

#     s1 = row['Street one']
#     s2 = row['Street two']

#     city = row['City']
#     zip = row['Postal code']

#     email = row['Email']

#     primary = True
#     if row['Primary?'] == "no":
#       primary = False

#     if (len(fn) > 0
#     or len(ln) > 0
#     or len(nn) > 0
#     or len(cn) > 0):

#       if (len(mp) > 0
#       or len(hp) > 0
#       or len(op) > 0):

#         if (len(s1) > 0
#         or len(s2) > 0):

#           if (len(city) > 0
#           or len(zip) > 0):

#             if len(email) > 0:
#               # print(row)
#               csvWriter.writerow(row)




# if re.search(r"^[^\w]*$", row["Last name"]):
#   if re.search(r"^[^\w]*$", row["Company name"]):
#     if len(row["Email"]) > 0:
#       # if not (row["Email"].__contains__("management")
#       # or row["Email"].__contains__("countyfl.gov")
#       # or row["Email"].__contains__("americashomeplace")
#       # or row["Email"].__contains__("snrservices")
#       # or row["Email"].__contains__(".fl.us")
#       # or row["Email"].__contains__("luntconstruction")
#       # or row["Email"].__contains__("nailedit")
#       # or row["Email"].__contains__("jettbuilders")
#       # or row["Email"].__contains__("aatayants")
#       # or row["Email"].__contains__("lzhang22")
#       # or row["Email"].__contains__("armcocabinets")
#       # or row["Email"].__contains__("rbabconstruction")
#       # or row["Email"].__contains__("regentadvisorsllc")
#       # or row["Email"].__contains__("hartsfieldconstruction")
#       # or row["Email"].__contains__("wordhomes")
#       # or row["Email"].__contains__("kentstraussmanagement")
#       # or row['Email'].__contains__('kw')):
#       # print(row["Email"])
#       # count += 1
#       csvWriter.writerow(row)

customers = createCustomerList()

print('Initial customer list')
for c in customerList:
  print(f"Customer {c['firstname']} {c['lastname']}")
  for l in c['locations']:
    print(f"\t{l}")

for row in rows:
  phones = makePhones(row)
  print(phones)
  # locations = makeLocations(row, next(c['locations'] for c in customerList if c['email'] == row['Email']))
  # print(locations)
  # customerList = makeCustomer(row, phones, locations, customerList)

# print('\nFinal customer list')
# for c in customerList:
#   print(f"Customer {c['firstname']} {c['lastname']}")
#   for l in c['locations']:
#     print(f"\t{l}")

# phones = makePhones(row)
# print(phones)
# locations = makeLocations(row, next(c['locations'] for c in customerList if c['email'] == 'joeadamscoast@comcast.net'))
# print(locations)

# l = temp[1]
# l = makeLocation(l['locations'])

# t = next(c for c in temp if c['firstname'] == 'lee')
# print(t['phone'])
# p, index = next(((index, p) for index, p in enumerate(t['phone']) if p['home'] == '902309f2g'), (None, None))
# if not index:
#   print(f"index: {index}")

# d, ii = next((d, ii) for ii, d in enumerate(temp) if d['firstname'] == 'lee')
# print(d)
# print(ii)

In [ ]:
from csv import DictReader


with open(csvFile, newline='') as input, open(jsonFile, 'w', encoding='utf-8') as output:
  csvReader = csv.DictReader(input)
  # jsonWriter = csv.DictWriter(output)

  customers = []
  count = 0

  for row in csvReader:
    email = row['Email']

    # Find if existing customer has same email (assume same customer)
    c, index = next(((c, index) for index, c in enumerate(customers) if c['email'] == email), (None, None))
    if c:
      del customers[index]
      
      # Fill in names, if blank
      if not c['firstname']:
        c['firstname'] = row['First name']
      if not c['lastname']:
        c['lastname'] = row['Last name']
      if not c['companyname']:
        c['companyname'] = row['Company name']
      if not c['nickname']:
        c['nickname'] = row['Nickname']
            
      phones = makePhones(row)
      for p1 in enumerate(row):
        index = next(((index, p) for p in phones if p == p1), None)
        if not index:
          continue
        else:
          del phones[index]


      location = makeLocation(row)
      index, l = next(((index, l) for index, l in enumerate(c['locations']) if l['zipcode'] == location['zipcode']), (None, None))
      if not index:

      if not index == None:
        

          # Search for same location:
          #   Search zip
          #     DNE?: Make location and add
          #   Search address
          #     DNE?: Make location and add

      customers.append(row)

# Need to search in object for matches and inject
    # if row1['Email']:
    #   locationCollection = []
    #   phoneCollection = []

    #   for row2 in csvReader:
    #     if row2['Email'] == row1['Email']:
    #       phoneCollection.append(makePhones(row2))
    #       locationCollection.append(makeLocation(row2))
        
    #     customers.append(makeCustomer(row1, phoneCollection, locationCollection))


    # if (len(fn) > 0
    # or len(ln) > 0
    # or len(nn) > 0
    # or len(cn) > 0):

    #   if (len(mp) > 0
    #   or len(hp) > 0
    #   or len(op) > 0):

    #     if (len(s1) > 0
    #     or len(s2) > 0):

    #       if (len(city) > 0
    #       or len(zip) > 0):

    #         if len(email) > 0:
    #             # print(row)
    #             newRow = {
    #               'firstname': fn,
    #               'lastname': ln,
    #               'companyname': cn,
    #               'nickname': nn,
    #               'phones': {
    #                 'mobile': mp,
    #                 'home': hp,
    #                 'office': op,
    #               },
    #               'locations': {
    #                 'address1': s1,
    #                 'address2': s2,
    #                 'city': city,
    #                 'state': 'FL',
    #                 'zipcode': zip,
    #                 'primary': primary,
    #               },
    #             }
    #             dataDict.append(newRow)
    #             continue

  jsonString = json.dumps(customers, indent=4)
  output.write(jsonString)

  print(count)